# Descripción
El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
# Librerías y modulos generales
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import catboost

In [2]:
# Cargamos dataset
df = pd.read_csv('car_data.csv')

In [3]:
# Revisamos muestra de dataset e info
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
322923,07/03/2016 11:50,5290,wagon,2003,manual,131,a4,150000,2,gasoline,audi,no,07/03/2016 00:00,0,71686,15/03/2016 05:16
91219,15/03/2016 22:52,12480,wagon,2005,manual,231,3er,150000,12,gasoline,bmw,no,15/03/2016 00:00,0,87549,16/03/2016 01:42
42317,01/04/2016 23:50,4500,bus,2006,manual,131,scenic,125000,1,gasoline,renault,no,01/04/2016 00:00,0,44579,06/04/2016 02:44
285345,11/03/2016 17:56,3995,small,2007,manual,80,corsa,150000,3,petrol,opel,no,11/03/2016 00:00,0,26670,05/04/2016 18:46
195950,29/03/2016 09:36,550,bus,2001,manual,107,scenic,150000,4,petrol,renault,yes,29/03/2016 00:00,0,84389,31/03/2016 03:15


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

### Formateo de columnas

In [5]:
# Atajo para renombrar columnas
df.columns + "'" + ": " + "'" + df.columns.str.lower()

Index(['DateCrawled': 'datecrawled', 'Price': 'price',
       'VehicleType': 'vehicletype', 'RegistrationYear': 'registrationyear',
       'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model',
       'Mileage': 'mileage', 'RegistrationMonth': 'registrationmonth',
       'FuelType': 'fueltype', 'Brand': 'brand', 'NotRepaired': 'notrepaired',
       'DateCreated': 'datecreated', 'NumberOfPictures': 'numberofpictures',
       'PostalCode': 'postalcode', 'LastSeen': 'lastseen'],
      dtype='object')

In [6]:
# Formateamos nombres de columnas
df = df.rename(columns={'DateCrawled': 'date_crawled', 'Price': 'price', 'VehicleType': 'vehicle_type',
                        'RegistrationYear': 'reg_year', 'Gearbox': 'gearbox', 'Power': 'power',
                        'Model': 'model', 'Mileage': 'mileage', 'RegistrationMonth': 'reg_month',
                        'FuelType': 'fuel_type', 'Brand': 'brand', 'NotRepaired': 'not_repaired',
                        'DateCreated': 'date_created', 'NumberOfPictures': 'number_of_pictures',
                        'PostalCode': 'postal_code', 'LastSeen': 'last_seen_date'})

In [7]:
# Realizamos prueba para saber si la fecha de descarga es la misma que la fecha de creación
df['date_crawled'] = pd.to_datetime(df['date_crawled'])
df['date_created'] = pd.to_datetime(df['date_created'])
df['last_seen_date'] = pd.to_datetime(df['last_seen_date'])

df[['date_crawled', 'date_created']][df['date_crawled'].dt.date !=
                                     df['date_created'].dt.date].head()

,date_crawled,date_created
42,2016-03-24 00:52:00,2016-03-23
47,2016-04-04 00:38:00,2016-03-04
67,2016-03-21 12:47:00,2016-09-02
99,2016-03-28 00:56:00,2016-03-27
178,2016-03-21 00:59:00,2016-03-20


Al saber que si hay fechas diferentes eliminaremos una columna, en este caso 'date_crawled' no nos sirve para nuesto análisis.

In [8]:
# Agregamos columnas necesarias
df['postal_region'] = df['postal_code'] // 1000

In [9]:
df.columns

Index(['date_crawled', 'price', 'vehicle_type', 'reg_year', 'gearbox', 'power',
       'model', 'mileage', 'reg_month', 'fuel_type', 'brand', 'not_repaired',
       'date_created', 'number_of_pictures', 'postal_code', 'last_seen_date',
       'postal_region'],
      dtype='object')

In [10]:
# Reordenamos columnas y eliminamos las que no aportan al análisis
selected_ordered_columns = ['date_created', 'price', 'reg_year', 'reg_month', 'mileage', 'brand',
                            'model', 'vehicle_type', 'gearbox', 'power', 'fuel_type', 'not_repaired',
                            'postal_region', 'last_seen_date']
df = df[selected_ordered_columns]

Se eliminaron las columnas 'date_crawled' (dato técnico) y 'number_of_pictures' (columnas todas en '0') dado que no aportan información necesaria.

### Tratamiento de filas duplicadas

In [11]:
# Investigamos filas duplicadas
print(f'Filas duplicadas: {df.duplicated().sum()}')
df[df.duplicated(keep=False)].sort_values(by='last_seen_date').head(6)

Filas duplicadas: 1082


,date_created,price,reg_year,reg_month,mileage,brand,model,vehicle_type,gearbox,power,fuel_type,not_repaired,postal_region,last_seen_date
237989,2016-03-19,16450,2009,6,60000,seat,leon,sedan,manual,241,petrol,no,66,2016-01-04 00:15:00
151423,2016-03-19,16450,2009,6,60000,seat,leon,sedan,manual,241,petrol,no,66,2016-01-04 00:15:00
196428,2016-03-28,8450,2007,1,150000,volkswagen,passat,wagon,auto,140,gasoline,no,18,2016-01-04 04:46:00
354151,2016-03-28,8450,2007,1,150000,volkswagen,passat,wagon,auto,140,gasoline,no,18,2016-01-04 04:46:00
234577,2016-03-24,2999,2000,2,150000,bmw,3er,wagon,auto,193,petrol,no,93,2016-01-04 05:45:00
224299,2016-03-24,2999,2000,2,150000,bmw,3er,wagon,auto,193,petrol,no,93,2016-01-04 05:45:00


In [12]:
# Eliminamos filas duplicadas
df = df.drop_duplicates()

### Tratamiento de valores ausentes

In [13]:
# Analizamos valores ausentes
df.isnull().sum()

date_created          0
price                 0
reg_year              0
reg_month             0
mileage               0
brand                 0
model             19691
vehicle_type      37463
gearbox           19821
power                 0
fuel_type         32874
not_repaired      71099
postal_region         0
last_seen_date        0
dtype: int64

In [14]:
# Hipótesis de diferencia de precio
print("Precio promedio por categoría:")
print(df.groupby('not_repaired')['price'].mean())

Precio promedio por categoría:
not_repaired
no     5295.742038
yes    1915.973464
Name: price, dtype: float64


In [15]:
# Valores ausentes con tratamiento distinto
df['not_repaired'].fillna('not_disclosed', inplace=True)

La columna 'not_repaired' se trató de manera diferente ya que tenemos la hipótesis de que la mayoría de los vendedores que no contestan este dato es porque 'sí' fueron reparados pero no quieren que el precio se vea castigado, como se puede observar en los datos de la celda anterior.

In [16]:
# Función para tratar valores ausentes regulares
def replace_unknown_null(df):
    """
    Reemplaza los valores ausentes por 'unknown' en columnas descritas.
    """
    columns = ['model', 'vehicle_type', 'gearbox', 'fuel_type']
    df[columns] = df[columns].replace(np.nan, 'unknown')
    return df


# Aplicamos función
df = replace_unknown_null(df)

# Comprobamos cambios
df.isnull().sum()

date_created      0
price             0
reg_year          0
reg_month         0
mileage           0
brand             0
model             0
vehicle_type      0
gearbox           0
power             0
fuel_type         0
not_repaired      0
postal_region     0
last_seen_date    0
dtype: int64

### Tratamiento de datos incongruentes

In [17]:
# Unificamos nombres de categorías
df['brand'] = df['brand'].replace('sonstige_autos', 'others')
df['fuel_type'] = df['fuel_type'].replace('petrol', 'gasoline')

In [18]:
print(
    f"Autos menores al año 1900: {df['reg_year'][df['reg_year'] > 2018].count()}")

Autos menores al año 1900: 130


In [19]:
# Análisis columna 'reg_year'
print(
    f"Autos menores al año 1900: {df['reg_year'][df['reg_year'] < 1900].count()}")
print(
    f"Autos mayores al año 2017: {df['reg_year'][df['reg_year'] > 2017].count()}")
# Eliminamos las filas con autos de año menores a 1900 y mayores al año 2017
df = df[(df['reg_year'] >= 1900) & (df['reg_year'] <= 2017)]

Autos menores al año 1900: 66
Autos mayores al año 2017: 4086


Eliminamos años incongruentes dado que es imposible tener autos con modelos de años antes de 1900 y despues del año de cuando se tomaron estos datos (2016).

In [20]:
# Las filas con precios menores a '20' euros no sirven para entrenar el modelo, lo sesgan
unclean_len = len(df['price'])
df = df[df['price'] > 20]
clean_len = len(df['price'])
print(f'Filas eliminadas: {unclean_len - clean_len}')

Filas eliminadas: 11973


In [21]:
# Relizamos imputación para los valores '0' en la columna 'power'

print(
    f'Datos con power = "0" ANTES de imputación: {len(df[df["power"] == 0])}')

# Creamos función de imputación


def impute_power_conditional(df, threshold=10):
    """
    Imputa valores de power = 0 usando estrategia condicional
    Si no encuentra grupo exacto, usa model como respaldo
    """
    df_copy = df.copy()

    # Estadísticas para grupos exactos (5 características)
    power_stats = df_copy[df_copy['power'] > 0].groupby(
        ['brand', 'model', 'vehicle_type', 'fuel_type', 'gearbox']
    )['power'].agg(['mean', 'median', 'count']).round(2)
    power_stats_filtered = power_stats[power_stats['count'] > 0]

    # Estadísticas de respaldo (model)
    backup_stats = df_copy[df_copy['power'] > 0].groupby('model'
                                                         )['power'].agg(['mean', 'median', 'count']).round(2)
    backup_stats_filtered = backup_stats[backup_stats['count'] > 0]

    # Aplicamos imputación
    for index, row in df_copy[df_copy['power'] == 0].iterrows():
        try:
            # Grupo exacto (todas las características)
            group_key = (row['brand'], row['model'], row['vehicle_type'],
                         row['fuel_type'], row['gearbox'])

            if group_key in power_stats_filtered.index:
                stats = power_stats_filtered.loc[group_key]
                mean_val = stats['mean']
                median_val = stats['median']
                difference = abs(mean_val - median_val)

                # Condicional para elegir media o mediana
                if difference > threshold:
                    impute_value = median_val
                else:
                    impute_value = mean_val

                df_copy.loc[index, 'power'] = impute_value

            else:
                # Respaldo (solo model)
                backup_key = row['model']

                if backup_key in backup_stats_filtered.index:
                    backup_stats_row = backup_stats_filtered.loc[backup_key]
                    backup_mean = backup_stats_row['mean']
                    backup_median = backup_stats_row['median']
                    backup_difference = abs(backup_mean - backup_median)

                    # Condicional para elegir media o mediana
                    if backup_difference > threshold:
                        impute_value = backup_median
                    else:
                        impute_value = backup_mean

                    df_copy.loc[index, 'power'] = impute_value

        except KeyError:
            # Si tampoco encuentra el respaldo, continuar sin imputar
            continue

    return df_copy


# Aplicamos imputación
df_clean = impute_power_conditional(df)
print(
    f'Datos con power = "0" DESPUES de imputación: {len(df_clean[df_clean["power"] == 0])}')

Datos con power = "0" ANTES de imputación: 34656
Datos con power = "0" DESPUES de imputación: 1


In [22]:
# Después de la prueba exitosa sobreescribimos el dataset original
df = df_clean

Realizamos imputación con media o mediana según sea el caso ya que la columna 'power' contenía más del 10% de datos en '0', y eso estaba sesgando nuestro modelo.

In [23]:
# Imputación para los valores 'unknown' en la columna 'model'
print(
    f'Datos con model = "unknown" ANTES de imputación: {len(df[df["model"] == "unknown"])}')


def impute_model_conditional(df, threshold=10):
    """
    Imputa valores de model = 'unknown' usando estrategia condicional
    Si no encuentra grupo exacto, usa brand como respaldo
    """
    df_copy = df.copy()

    # Estadísticas para grupos exactos (4 características)
    model_stats = df_copy[df_copy['model'] != 'unknown'].groupby(
        ['brand', 'vehicle_type', 'fuel_type', 'gearbox']
    )['model'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown')

    # Estadísticas de respaldo (brand + vehicle type)
    backup_stats = df_copy[df_copy['model'] != 'unknown'].groupby(['brand', 'vehicle_type']
                                                                  )['model'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown')

    # Aplicamos imputación
    for index, row in df_copy[df_copy['model'] == 'unknown'].iterrows():
        try:
            # Grupo exacto (todas las características)
            group_key = (row['brand'], row['vehicle_type'], row['fuel_type'],
                         row['gearbox'])

            if group_key in model_stats.index:
                impute_value = model_stats.loc[group_key]
                df_copy.loc[index, 'model'] = impute_value

            else:
                # Respaldo (brand + vehicle type)
                backup_key = (row['brand'], row['vehicle_type'])

                if backup_key in backup_stats.index:
                    impute_value = backup_stats.loc[backup_key]
                    df_copy.loc[index, 'model'] = impute_value

        except KeyError:
            # Si tampoco encuentra el respaldo, continuar sin imputar
            continue

    return df_copy


# Aplicamos imputación
df_clean = impute_model_conditional(df)
print(
    f'Datos con model = "unknown" DESPUES de imputación: {len(df_clean[df_clean["model"] == "unknown"])}')

Datos con model = "unknown" ANTES de imputación: 16614
Datos con model = "unknown" DESPUES de imputación: 2678


In [24]:
# Después de la prueba exitosa sobreescribimos el dataset original
df = df_clean

Realizamos imputación para model = 'unknown' con las características 'brand', 'vehicle_type', 'fuel_type' y 'gearbox' ya que tenemos más de 16 mil filas que no sabemos el modelo.

In [25]:
# Cambio de tipos de datos
df['price'] = df['price'].astype(float)
categorical_columns = ['brand', 'model', 'vehicle_type',
                       'gearbox', 'fuel_type', 'not_repaired', 'postal_region']

for col in categorical_columns:
    df[col] = df[col].astype('category')

# Comprobar cambios
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337162 entries, 0 to 354368
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date_created    337162 non-null  datetime64[ns]
 1   price           337162 non-null  float64       
 2   reg_year        337162 non-null  int64         
 3   reg_month       337162 non-null  int64         
 4   mileage         337162 non-null  int64         
 5   brand           337162 non-null  category      
 6   model           337162 non-null  category      
 7   vehicle_type    337162 non-null  category      
 8   gearbox         337162 non-null  category      
 9   power           337162 non-null  float64       
 10  fuel_type       337162 non-null  category      
 11  not_repaired    337162 non-null  category      
 12  postal_region   337162 non-null  category      
 13  last_seen_date  337162 non-null  datetime64[ns]
dtypes: category(7), datetime64[ns](2), f

## Entrenamiento de los modelos 

In [26]:
# Segmentamos los datos en entrenamiento y prueba
import time
from sklearn.model_selection import train_test_split

random = 23451
df_train, df_test = train_test_split(df, test_size=0.30, random_state=random)

features_train = df_train.drop(
    ['date_created', 'price', 'last_seen_date'], axis=1)
target_train = df_train['price']
features_test = df_test.drop(
    ['date_created', 'price', 'last_seen_date'], axis=1)
target_test = df_test['price']

In [27]:
%%time
# PRIMEROS HIPERPARÁMETROS
# Entrenamos modelo con LightGBM 
import lightgbm as lgb

# Definimos columnas categóricas
lgb_features = ['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type', 'not_repaired', 'postal_region']

# Elegimos hiperparámetros
model_lgb_first = lgb.LGBMRegressor(n_estimators=500,
                              num_leaves=31,
                              learning_rate=0.1,
                              categorical_feature=lgb_features,
                              metric='RMSE',
                              verbose=-1
                              )

# Entrenamos modelo
start_time = time.time()
model_lgb_first.fit(features_train, target_train)
end_time = time.time()
training_time_lgb_first = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_lgb_first = model_lgb_first.predict(features_test)
end_time = time.time()
pred_time_lgb_first = end_time - start_time


c:\Users\lans_\anaconda3\envs\tripleten\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\lans_\anaconda3\envs\tripleten\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\lans_\anaconda3\envs\tripleten\lib\subprocess.py", line 493, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\lans_\anaconda3\envs\tripleten\lib\subprocess.py", line 858, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\lans_\anaconda3\envs\tripleten\lib\subprocess.py", line 1327, in _execute_child
    hp, ht, pid, tid =

CPU times: total: 20 s
Wall time: 1.45 s


In [28]:
%%time
# SEGUNDOS HIPERPARÁMETROS
# Entrenamos modelo con LightGBM
import lightgbm as lgb

# Definimos columnas categóricas
lgb_features = ['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type', 'not_repaired', 'postal_region']

# Elegimos hiperparámetros
model_lgb_second = lgb.LGBMRegressor(n_estimators=700,
                              num_leaves=50,
                              learning_rate=0.15,
                              categorical_feature=lgb_features,
                              metric='RMSE',
                              verbose=-1
                              )

# Entrenamos modelo
start_time = time.time()
model_lgb_second.fit(features_train, target_train)
end_time = time.time()
training_time_lgb_second = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_lgb_second = model_lgb_second.predict(features_test)
end_time = time.time()
pred_time_lgb_second = end_time - start_time


CPU times: total: 34.1 s
Wall time: 2.32 s


In [29]:
%%time
# Entrenamos modelo con CatBoost
from catboost import CatBoostRegressor

# Definimos columnas categóricas
cat_features = ['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type', 'not_repaired', 'postal_region']

# Elegimos hiperparámetros
model_catb = CatBoostRegressor(random_seed=random,
                               iterations=500,
                               depth=6,
                              learning_rate=0.1,
                              loss_function='Quantile:alpha=0.4',
                              eval_metric='RMSE',
                               cat_features=cat_features,
                               verbose=100
                              )

# Entrenamos modelo
start_time = time.time()
model_catb.fit(features_train, target_train)
end_time = time.time()
training_time_catb = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_catb = model_catb.predict(features_test)
end_time = time.time()
pred_time_catb = end_time - start_time


0:	learn: 4954.9877069	total: 240ms	remaining: 1m 59s
100:	learn: 1982.5875471	total: 11s	remaining: 43.3s
200:	learn: 1882.1630459	total: 21.1s	remaining: 31.4s
300:	learn: 1828.8324621	total: 31.5s	remaining: 20.8s
400:	learn: 1800.7221537	total: 41.9s	remaining: 10.3s
499:	learn: 1780.4813593	total: 52.4s	remaining: 0us
CPU times: total: 6min 44s
Wall time: 52.9 s


In [30]:
%%time
# Entrenamos modelo con XGboost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Codificamos las variables categóricas
features_train_enc = features_train.copy()
features_test_enc = features_test.copy()

categorical_cols = ['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type', 'not_repaired', 'postal_region']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Ajustamos con todos los datos (train + test) para evitar problemas
    all_values = pd.concat([features_train_enc[col], features_test_enc[col]])
    le.fit(all_values)
     
    features_train_enc[col] = le.transform(features_train_enc[col])
    features_test_enc[col] = le.transform(features_test_enc[col])
    label_encoders[col] = le


# Elegimos hiperparámetros
model_xgb = xgb.XGBRegressor(random_state=random,
                             iterations=500,
                              learning_rate=0.1,
                              max_leaves=31,
                              eval_metric='rmse',
                               verbosity=0
                              )

# Entrenamos modelo
start_time = time.time()
model_xgb.fit(features_train_enc, target_train)
end_time = time.time()
training_time_xgb = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_xgb = model_xgb.predict(features_test_enc)
end_time = time.time()
pred_time_xgb = end_time - start_time


CPU times: total: 5.14 s
Wall time: 705 ms


In [31]:
%%time
# PRIMEROS HIPERPARÁMETROS
# Entrenamos modelo de Bosque aleatorio
from sklearn.ensemble import RandomForestRegressor

# Elegimos hiperparámetros
model_forest_first = RandomForestRegressor(random_state=random,
                                     n_estimators=500,
                                     max_depth=None,
                                     max_leaf_nodes=31,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     n_jobs=-1,
                                     verbose=0
                                    )

# Entrenamos modelo
start_time = time.time()
model_forest_first.fit(features_train_enc, target_train)
end_time = time.time()
training_time_forest_first = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_forest_first = model_forest_first.predict(features_test_enc)
end_time = time.time()
pred_time_forest_first = end_time - start_time


CPU times: total: 3min 37s
Wall time: 15.2 s


In [32]:
%%time
# SEGUNDOS HIPERPARÁMETROS
# Entrenamos modelo de Bosque aleatorio
from sklearn.ensemble import RandomForestRegressor

# Elegimos hiperparámetros
model_forest_second = RandomForestRegressor(random_state=random,
                                     n_estimators=1000,
                                     max_depth=None,
                                     max_leaf_nodes=50,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     n_jobs=-1,
                                     verbose=0
                                    )

# Entrenamos modelo
start_time = time.time()
model_forest_second.fit(features_train_enc, target_train)
end_time = time.time()
training_time_forest_second = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_forest_second = model_forest_second.predict(features_test_enc)
end_time = time.time()
pred_time_forest_second = end_time - start_time


CPU times: total: 7min 46s
Wall time: 31.1 s


In [33]:
%%time
# Entrenamos modelo de Árbol de decisión
from sklearn.tree import DecisionTreeRegressor

# Elegimos hiperparámetros
model_tree = DecisionTreeRegressor(random_state=random,
                                   max_depth=None,
                                   max_leaf_nodes=31,
                                   min_samples_split=2,
                                   min_samples_leaf=1,
                                   criterion='squared_error'
                                  )

# Entrenamos modelo
start_time = time.time()
model_tree.fit(features_train_enc, target_train)
end_time = time.time()
training_time_tree = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_tree = model_tree.predict(features_test_enc)
end_time = time.time()
pred_time_tree = end_time - start_time


CPU times: total: 359 ms
Wall time: 352 ms


In [34]:
%%time
# Entrenamos modelo de Regresión lineal
from sklearn.linear_model import LinearRegression

# Elegimos hiperparámetros
model_lr = LinearRegression()

# Entrenamos modelo
start_time = time.time()
model_lr.fit(features_train_enc, target_train)
end_time = time.time()
training_time_lr = end_time - start_time

# Predecimos modelo
start_time = time.time()
pred_lr = model_lr.predict(features_test_enc)
end_time = time.time()
pred_time_lr = end_time - start_time


CPU times: total: 62.5 ms
Wall time: 62.7 ms


## Análisis de modelos

In [35]:
# Cálculos de RMSE por modelo
from sklearn.metrics import mean_squared_error

rmse_lgb_first = mean_squared_error(target_test, pred_lgb_first, squared=False)
rmse_lgb_second = mean_squared_error(
    target_test, pred_lgb_second, squared=False)
rmse_catb = mean_squared_error(target_test, pred_catb, squared=False)
rmse_xgb = mean_squared_error(target_test, pred_xgb, squared=False)
rmse_forest_first = mean_squared_error(
    target_test, pred_forest_first, squared=False)
rmse_forest_second = mean_squared_error(
    target_test, pred_forest_second, squared=False)
rmse_tree = mean_squared_error(target_test, pred_tree, squared=False)
rmse_lr = mean_squared_error(target_test, pred_lr, squared=False)

In [36]:
# Presentación de métricas
print(f'MODELO LIGHTGBM:')
print(f' Primeros parámetros:')
print(f'  RECM: {rmse_lgb_first.round(2)}\n  Tiempo de entrenamiento: {round(training_time_lgb_first, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_lgb_first, 4)} segundos.')
print(f' Segundos parámetros:')
print(f'  RECM: {rmse_lgb_second.round(2)}\n  Tiempo de entrenamiento: {round(training_time_lgb_second, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_lgb_second, 4)} segundos.\n')
print(f'MODELO CATBOOST:')
print(f'  RECM: {rmse_catb.round(2)}\n  Tiempo de entrenamiento: {round(training_time_catb, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_catb, 4)} segundos.\n')
print(f'MODELO XGBOOST:')
print(f'  RECM: {rmse_xgb.round(2)}\n  Tiempo de entrenamiento: {round(training_time_xgb, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_xgb, 4)} segundos.\n')
print(f'MODELO RANDOM FOREST:')
print(f' Primeros parámetros:')
print(f'  RECM: {rmse_forest_first.round(2)}\n  Tiempo de entrenamiento: {round(training_time_forest_first, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_forest_first, 4)} segundos.')
print(f' Segundos parámetros:')
print(f'  RECM: {rmse_forest_second.round(2)}\n  Tiempo de entrenamiento: {round(training_time_forest_second, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_forest_second, 4)} segundos.\n')
print(f'MODELO ÁRBOL DE DECISIÓN:')
print(f'  RECM: {rmse_tree.round(2)}\n  Tiempo de entrenamiento: {round(training_time_tree, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_tree, 4)} segundos.\n')
print(f'MODELO DE REGRESIÓN LINEAL:')
print(f'  RECM: {rmse_lr.round(2)}\n  Tiempo de entrenamiento: {round(training_time_lr, 4)} segundos.\n  Tiempo de predicción: {round(pred_time_lr, 4)} segundos.\n')

MODELO LIGHTGBM:
 Primeros parámetros:
  RECM: 1620.44
  Tiempo de entrenamiento: 1.1788 segundos.
  Tiempo de predicción: 0.234 segundos.
 Segundos parámetros:
  RECM: 1601.24
  Tiempo de entrenamiento: 1.9686 segundos.
  Tiempo de predicción: 0.3472 segundos.

MODELO CATBOOST:
  RECM: 1803.98
  Tiempo de entrenamiento: 52.8193 segundos.
  Tiempo de predicción: 0.0526 segundos.

MODELO XGBOOST:
  RECM: 1799.03
  Tiempo de entrenamiento: 0.3211 segundos.
  Tiempo de predicción: 0.0336 segundos.

MODELO RANDOM FOREST:
 Primeros parámetros:
  RECM: 2376.1
  Tiempo de entrenamiento: 14.8361 segundos.
  Tiempo de predicción: 0.1737 segundos.
 Segundos parámetros:
  RECM: 2271.27
  Tiempo de entrenamiento: 30.8078 segundos.
  Tiempo de predicción: 0.3184 segundos.

MODELO ÁRBOL DE DECISIÓN:
  RECM: 2403.4
  Tiempo de entrenamiento: 0.3458 segundos.
  Tiempo de predicción: 0.006 segundos.

MODELO DE REGRESIÓN LINEAL:
  RECM: 3559.26
  Tiempo de entrenamiento: 0.0587 segundos.
  Tiempo de pre

## Conclusión

Todos los modelos de potenciación del gradiente son más efeiciente en cuanto que los demás modelos. En resumen el modelo de potenciación del gradiente LightGBM utilizando los segundos parámetros es el más eficiente en cuanto al tiempo de entrenamiento y calidad de la predicción en comparación con los otros modelos.  

El modelo de potenciación del gradiente es **LightGBM** es el mejor en las 3 métricas solicitadas:
- La calidad de predicción del modelo.
- El tiempo utilizado en entrenarlo.
- La velocidad para predecir.

In [37]:
# Versiones de librerías
import sys
print("Versiones de las librerías:")
print(f"python=={sys.version}")
print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"seaborn=={sns.__version__}")
print(f"sklearn=={sklearn.__version__}")
print(f"lightgbm=={lgb.__version__}")
print(f"catboost=={catboost.__version__}")
print(f"xgboost=={xgb.__version__}")

Versiones de las librerías:
python==3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
numpy==1.23.5
pandas==1.2.4
seaborn==0.13.2
sklearn==1.3.2
lightgbm==4.4.0
catboost==1.2.5
xgboost==2.1.1
